# Setup

Install dependencies and check PyTorch and GPU.

In [ ]:
!pip install ultralytics

# 1. Inference

With yolo setup, we'll do a test inference on some images in our data set. We'll be using the `yolov8n.pt` model to see how well it identifies airplanes and helicopters. You should see mostly failures to detect objects and bad classifications like 'boat' and 'kite'.

In [ ]:
%cd /opt/app-root/src/
!yolo predict model=yolov8n.pt source=datasets/training/valid/images exist_ok=True

## Sample Images
Let's see how well the default model did with some random selections. You can run this cell multiple times.

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Directory path containing the images
directory_path = "runs/detect/predict"

# Get a list of all image files in the directory
image_files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Randomly select four images
selected_images = random.sample(image_files, 4)

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Iterate over the selected images and display them
for i, image_file in enumerate(selected_images):
    # Construct the image file path
    image_path = os.path.join(directory_path, image_file)

    # Load and display the image
    image = mpimg.imread(image_path)
    axes[i // 2, i % 2].imshow(image)
    axes[i // 2, i % 2].axis("off")

# Adjust the layout of subplots
plt.tight_layout()

# Display the figure
plt.show()

# 2. Train
Now we'll train a new model with our flying things dataset. The following will be used to kick it off:

```shell
yolo train model=yolov8n.pt        # use the default yolov8 nano model as a baseline
           batch=2                 # batch size determines the amount of concurrent processing
           epochs=100              # the number of times through the entire dataset
           data=flyingthings.yaml  # file containing the class id and names and the directories for test, train, and valid
           exist_ok=True           # allow results to be overwritten
```

In [ ]:
%cd /opt/app-root/src/
!yolo train model=yolov8n.pt batch=2 epochs=1 data=flyingthings.yaml exist_ok=True

## Results
Once training is complete, we can look at the results. Here are some graphs indicating accuracy over the the course of epoch runs.

In [ ]:
from PIL import Image
import IPython.display as display

# Set the paths to the image files
image_paths = [
    'runs/detect/train/results.png',
    'runs/detect/train/F1_curve.png',
    'runs/detect/train/P_curve.png',
    'runs/detect/train/R_curve.png'
]

# Create a list to hold the Image objects
images = []

# Open the images and append them to the list
for path in image_paths:
    image = Image.open(path)
    images.append(image)

# Display the images
for image in images:
    display.display(image)

# 3 Inference with new model
Now we'll run the same images we used earlier with our new model to see how much better it does at identifying airplanes and helicopters.

In [ ]:
%cd /opt/app-root/src/
!yolo predict model=runs/detect/train/weights/best.pt source=datasets/training/valid/images exist_ok=True

## Sample Images
Here are some random images from inferencing with our new model. You can run this cell multiple times.

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Directory path containing the images
directory_path = "runs/detect/predict"

# Get a list of all image files in the directory
image_files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Randomly select four images
selected_images = random.sample(image_files, 4)

# Create a figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Iterate over the selected images and display them
for i, image_file in enumerate(selected_images):
    # Construct the image file path
    image_path = os.path.join(directory_path, image_file)

    # Load and display the image
    image = mpimg.imread(image_path)
    axes[i // 2, i % 2].imshow(image)
    axes[i // 2, i % 2].axis("off")

# Adjust the layout of subplots
plt.tight_layout()

# Display the figure
plt.show()

# 4. Save our new model
Now that we have a model that's better at predicting helis and planes, we will save it to our bucket. As we improve our model with additional training and other adjustments, they will be saved and versioned allowing us to use previous versions for comparrisons or other analysis.

In [ ]:
import os
import minio
import urllib3

# Disable SSL certificate verification
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up MinIO client with insecure HTTPS
endpoint_str = os.environ.get('AWS_S3_ENDPOINT')
minio_endpoint = endpoint_str.lstrip('https://')
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
minio_client = minio.Minio(
    endpoint=minio_endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=True,
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE', assert_hostname=False)
)


# Set the bucket name and file path
bucket_name = os.environ.get('AWS_S3_BUCKET')
file_path = 'runs/detect/train/weights/best.pt'

# Set the object name (the name under which the file will be stored in the bucket)
object_name = 'flyingthings.pt'

# Upload the file
minio_client.fput_object(bucket_name, object_name, file_path)

print(f"The file '{object_name}' has been uploaded to the '{bucket_name}' bucket.")
